# Create Monthly Task Reports in Smartsheet


In [ ]:
import os
import json
import logging
from datetime import datetime, timedelta
from dateutil import parser
from typing import Dict, List

import pandas as pd
import numpy as np
import prefect
from box import Box

import smartsheet

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient

from mondaydotcom_utils.formatted_value import (
    FormattedValue,
    get_col_defs,
    get_items_by_board,
)

import scrapbook as sb
import dotenv

from prefect import Flow, Parameter, task, unmapped
from prefect.executors import LocalDaskExecutor, LocalExecutor

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
TASKS_BOARD_ID = "1883170887"
AGREEMENTS_BOARD_ID = "1882423671"
PROJECTS_BOARD_ID = "1882404316"
ACCOUNTS_BOARD_ID = "1882424009"

PROJECT_TASK_TIME_BOARD_ID = "2398200403"

posted_tasks_id = 3567675495475076

In [ ]:
environment = "dev"

# change these or set as papermill parameters to report on year and month
year_for_report = 2022
month_for_report = 4

In [ ]:
#check the environment vars for secrets

env_file = f".env-{environment}"
logger.info("Loading the .env file from %s", env_file)
dotenv.load_dotenv(dotenv.find_dotenv(env_file))

assert os.environ.get("MONDAY_KEY"), f"MONDAY_KEY not found in {env_file}"
assert os.environ.get("SMARTSHEET_KEY"), f"SMARTSHEET_KEY not found in {env_file}"

In [ ]:
# connect monday client
conn = MondayClient(os.environ.get("MONDAY_KEY"))

In [ ]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(os.environ.get("SMARTSHEET_KEY"))
ss_client.errors_as_exceptions(True)

In [ ]:
# bug between ProdPerfect and MDC's API: https://github.com/ProdPerfect/monday/issues/57
from monday.resources.base import BaseResource

query = """query
    {
        users () {
            id
            name
            email
            enabled
        }
    }"""
query

In [ ]:
base_resource = BaseResource(os.environ.get("MONDAY_KEY"))
users = base_resource._query(query)["data"]["users"]

In [ ]:
users_df = pd.DataFrame(users).set_index("id")
users_df.head()

In [ ]:
accounts_df = get_items_by_board(conn, ACCOUNTS_BOARD_ID).fillna("")

accounts_df.rename(
    columns={
        "monday_id": "account_id",
        "monday_name": "Client Name",
        "No Bill__checked": "No Bill",
    },
    inplace=True,
)

accounts_df.drop(
    columns=[
        "Contacts",
        "Item ID",
        "Subitems",
        "Notes",
        "Customer Projects",
        "Agreements",
        "Type__text",
        "Type__changed_at",
        "Type",
        "No Bill__changed_at",
        "Item ID__default_formatter",
    ],
    inplace=True,
    errors="ignore",
)

accounts_df

In [ ]:
projects_df = get_items_by_board(conn, PROJECTS_BOARD_ID).fillna("")

projects_df.rename(
    columns={
        "monday_id": "project_id",
        "monday_name": "Project Title",
        "Project Lifecycle__text": "Project Lifecycle",
        "Account": "account_id",
    },
    inplace=True,
)

projects_df.drop(
    columns=[
        "Project Tasks",
        "Subitems",
        "Project Contacts",
        "SET Resource",
        "Timeline",
        "Customer Source",
        "Dependency",
        "Date Added",
        "Timeline Days",
        "Item ID",
        "Project Health",
        "Notes",
        "Agreements",
        "Project Health__text",
        "Project Health__changed_at",
        "Date Added__default_formatter",
        "Item ID__default_formatter",
        "Tasks Status__mirror",
        "Project Lifecycle__changed_at",
        "Repo Description__mirror",
        "Timeline__to",
        "Timeline__from",
        "Timeline__changed_at",
    ],
    inplace=True,
    errors="ignore",
)

projects_df = projects_df.explode(["account_id"], ignore_index=True)
projects_df

In [ ]:
# add the account to the projects
projects_df = pd.merge(projects_df, accounts_df, how="left", on="account_id").drop(
    columns=["account_id"]
)
projects_df

In [ ]:
# only getting done tasks
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID, "status", "Done")

# Only include Ready tasks
tasks_df = tasks_df.loc[
    tasks_df["Integration Message"].str.startswith("Ready", na=False)
]

tasks_df.rename(
    columns={
        "monday_id": "task_id",
        "monday_name": "Task Name",
        "Customer Project": "project_id",
    },
    inplace=True,
)

tasks_df.drop(
    columns=[
        "Subtasks",
        "Customer Repos",
        "Billing Agreement",
        "Timeline Hours (Estimated)__formula",
        "Timeline__visualization_type",
        "Actual Time__running",
        "Timeline__to",
        "Timeline__from",
        "Timeline__changed_at",
        "Timeline Days",
        "Total Actual Hours__formula",
        "Date Added__default_formatter",
        "Timeline",
        "Actual Time__startDate",
        "Actual Time__changed_at",
        "Actual Time",
        "Date Added",
        "Dependencies",
    ],
    inplace=True,
    errors="ignore",
)

tasks_df = tasks_df.explode(["project_id"], ignore_index=True)
tasks_df.head()

Validate takes an individual record and checks it against rules, and creates multiple task records where required.

If actual hours is used, then the number of owners dictates the number of journal records. E.g., actual hours = 15, with 3 owners, yields three journal entries at 5 each (actual hours / owner count).

In [ ]:
records = tasks_df.to_dict(orient="records")

In [ ]:
# check for actual hours records
owner_records = []

for record in records:

    actual_hours = record["Actual Hours"]
    owners_list = record["Owner"] if isinstance(record["Owner"], list) else []
    len_owners_list = len(owners_list)
    date_completed = record["Date Completed"]

    if not np.isnan(actual_hours):
        # split the hours up between the owners
        for owner in owners_list:
            new_rec = record.copy()

            # overwrite the owner
            new_rec["Owner"] = users_df.loc[owner["id"]]["name"]

            # divide the task time
            new_rec["hours"] = actual_hours / len_owners_list

            # get the task time from date completed... or fallback on the status
            if record.get("Date Completed") and record["Date Completed"]:
                new_rec["Task Completed"] = parser.parse(
                    f"{date_completed} 00:00:00+00:00"
                )
            else:
                new_rec["Task Completed"] = parser.parse(record["Status__changed_at"])

            new_rec["integration_state_rule"] = "hours_split_between_owners"
            owner_records.append(new_rec)

owner_records_df = pd.DataFrame(owner_records)
owner_records_df.drop(
    columns=[
        "Actual Time__additional_value",
        "Actual Time__duration",
        "Status__changed_at",
        "Actual Hours",
    ],
    inplace=True,
)
owner_records_df.rename(columns={"Status__text": "Status"}, inplace=True)
owner_records_df

In [ ]:
# run through the records, looking for sessions, and if found,
# create new journal tasks for each one
session_records = []

for record in records:

    if isinstance(record["Actual Time__additional_value"], list):
        sessions_list = record["Actual Time__additional_value"]
    else:
        sessions_list = []

    date_completed = record["Date Completed"]
    actual_hours = record["Actual Time__duration"] / 60 / 60

    # multiply the number of tasks by sessions
    for session in sessions_list:
        new_rec = record.copy()
        new_rec["Owner"] = users_df.loc[session["ended_user_id"]]["name"]

        start_date = parser.parse(session["started_at"])
        end_date = parser.parse(session["ended_at"])

        # take the difference between the two dates as hours
        difference = end_date - start_date
        new_rec["hours"] = difference.total_seconds() / 60 / 60

        # get the task time from date completed... or fallback on the status
        # we could also use the ended_at for each session, but the risk is
        # we might post a session that should have been posted against the wrong month.
        if record.get("Date Completed") and record["Date Completed"]:
            new_rec["Task Completed"] = parser.parse(f"{date_completed} 00:00:00+00:00")
        else:
            new_rec["Task Completed"] = parser.parse(status_json["Status__changed_at"])

    new_rec["integration_state_rule"] = "hours_from_session_records"
    session_records.append(new_rec)

session_records_df = pd.DataFrame(session_records)
session_records_df.drop(
    columns=[
        "Actual Time__additional_value",
        "Actual Time__duration",
        "Status__changed_at",
        "Actual Hours",
    ],
    inplace=True,
)
session_records_df.rename(columns={"Status__text": "Status"}, inplace=True)
session_records_df.sort_values(by="Task Name")

In [ ]:
# merge the owner and session records
journal_task_df = pd.concat([owner_records_df, session_records_df])
journal_task_df

Break down the Monday.com session items into individual "journal tasks".

In [ ]:
# convert to a datetime... a bit crude for filtering by year and month
journal_task_df["task_end_date"] = pd.to_datetime(journal_task_df["Date Completed"])
journal_task_df["task_end_year"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).year
journal_task_df["task_end_month"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).month

journal_task_df.head()

In [ ]:
# only interested in this month's tasks
mask = (journal_task_df["task_end_year"] == year_for_report) & (
    journal_task_df["task_end_month"] == month_for_report
)
journal_task_df = journal_task_df.loc[mask].copy()

# Some of this helped build the record, some is just mirror or lookup gak.
journal_task_df.drop(
    columns=[
        "Actual Hours",
        "Actual Time",
        "Customer Repos",
        "Date Added",
        "Date Completed",
        "Dependencies",
        "Integration Message",
        "Subtasks",
        "Timeline",
        "Total Actual Hours (formula)",
        "Total Duration Hours",
        "Time Sessions",
        "Status",
        "Timeline Hours (Estimated) (formula)",
        "Project Lifecycle (mirror)",
        "Billing Agreement",
        "Timeline Days",
        "task_status",
    ],
    inplace=True,
    errors="ignore",
)

journal_task_df

Finally merge the tasks and projects together for a final task list.

In [ ]:
def month_end_date(year, month):
    """Calculate the month end date given a year and month."""
    month += 1
    if month == 13:
        month = 1
        year += 1

    tempdate = datetime.strptime(f"{year}-{month}-1", "%Y-%m-%d")
    return (tempdate - timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:
def month_end_me(row):
    return month_end_date(row["task_end_year"], row["task_end_month"])

In [ ]:
df = pd.merge(
    journal_task_df,
    projects_df,
    how="left",
    on="project_id",
)

# create a month-ending date column
month_end_dates = df.apply(month_end_me, axis=1)
if len(month_end_dates) > 0:
    df["Month Ending Date"] = month_end_dates
else:
    # going through the motions now so the rest of the notebook will run
    df["Month Ending Date"] = None

df.rename(
    columns={
        "monday_id_x": "monday_id",
        "monday_id_y": "project_id",
        "Title_x": "Title",
        "Title_y": "Project Title",
        "Notes_x": "Notes",
        "Notes_y": "Project Notes",
        "hours": "Hours",
        "Owner": "Resource",
    },
    inplace=True,
)

df.drop(
    columns=[
        "project_id",
        "monday_id",
        "task_end_year",
        "task_end_month",
        "Account",
    ],
    inplace=True,
    errors="ignore",
)

df

In [ ]:
# create a group by report and post to SE Project/Grant Time smartsheet
report_df = (
    df.groupby(["Client Name", "Project Title", "Resource"])
    .agg(
        {
            "Hours": "sum",
            "Month Ending Date": "first",
            "Grant Number": "first",
            "No Bill": "first",
        }
    )
    .reset_index()
)
report_df

In [ ]:
def filter_tasks(df, client, project, resource, month_end_date):
    """Get the journal tasks based on the details we'll send to smartsheet."""
    return df[
        (
            (df["Client Name"] == client)
            & (df["Project Title"] == project)
            & (df["Resource"] == resource)
            & (df["Month Ending Date"] == month_end_date)
        )
    ]

Now, Smartsheet's turn?

In [ ]:
# smartsheet can have duplicate sheet names,
# so best not to rely on them
time_sheet_id = 3567675495475076
time_sheet = ss_client.Sheets.get_sheet(time_sheet_id)

In [ ]:
# break down the cell IDs into a quick lookup box
cell_ids = {}
for column in time_sheet.columns:
    my_column = column.to_dict()
    cell_ids[my_column["title"]] = my_column["id"]
cell_ids

Add the records to Smartsheet

In [ ]:
rows = []
for k, v in report_df.to_dict("index").items():

    # only post billable projects
    if not v["No Bill"]:

        row = ss_client.models.row.Row()

        row.cells.append(
            {"column_id": cell_ids["Account/Client"], "value": v["Client Name"]}
        )
        if v.get("Grant Number"):
            row.cells.append(
                {"column_id": cell_ids["Grant Proposal #"], "value": v["Grant Number"]}
            )
        row.cells.append(
            {"column_id": cell_ids["Project Title"], "value": v["Project Title"]}
        )
        row.cells.append(
            {"column_id": cell_ids["Month-end Date"], "value": v["Month Ending Date"]}
        )
        row.cells.append(
            {"column_id": cell_ids["Completed Hours"], "value": v["Hours"]}
        )
        row.cells.append({"column_id": cell_ids["Resource"], "value": v["Resource"]})

        row.to_bottom = True
        rows.append(row)

In [ ]:
result = None
if rows:
    logger.info("Adding %s posted rows to SmartSheet", len(rows))
    result = ss_client.Sheets.add_rows(time_sheet_id, rows)

result

In [ ]:
# get the row ids and create a Series
if result:
    my_list = []
    for row in result.to_dict()["data"]:
        my_list.append(row["id"])

    row_series = pd.Series(my_list, name="row_id", dtype=np.int64)
    row_series

    report_df = pd.concat([report_df, row_series], axis=1)

report_df

In [ ]:
# create a dictionary to make attaching the files easier
new_dict = {}

if report_df.get("row_id"):

    for k, v in report_df.to_dict("index").items():
        row_id = v["row_id"]

        # add the filtered tasks to a list
        new_dict[row_id] = filter_tasks(
            df,
            v["Client Name"],
            v["Project Title"],
            v["Resource"],
            v["Month Ending Date"],
        )

new_dict

In [ ]:
if not os.path.exists("_cache"):
    os.mkdir("_cache")

In [ ]:
# attach file to each record
for k, v in new_dict.items():
    filename = os.path.join("_cache", f"{k}.csv")

    # save file
    v.to_csv(filename, index=False)

    with open(filename, "r") as f:
        ss_client.Attachments.attach_file_to_row(time_sheet_id, k, f)

In [ ]:
billable_posted = False
for k, v in df.to_dict("index").items():
    if v["No Bill"]:
        result = conn.items.change_item_value(
            TASKS_BOARD_ID,
            v["task_id"],
            "text01",
            f"Posted - No Bill - {datetime.now()}",
        )
    else:
        result = conn.items.change_item_value(
            TASKS_BOARD_ID, v["task_id"], "text01", f"Posted - {datetime.now()}"
        )
        billable_posted = True

Delete any estimates

In [ ]:
# delete the estimates only
result = None
# only do this if we had some monthly tasks to report
if billable_posted and len(report_df) > 0:
    result = ss_client.Sheets.get_sheet(time_sheet_id, filter_id=5850658663360388)
    rows_to_delete = [x["id"] for x in result.to_dict()["rows"] if not x["filteredOut"]]
    if rows_to_delete:
        result = ss_client.Sheets.delete_rows(posted_tasks_id, rows_to_delete)
result